# Import Packages

In [5]:
import numpy as np
import pandas as pd
import glob
import os

# Define Dynamic Parameters

In [6]:
trend_days = 3
trend_file_name = 'GoogleTrendsAdjusted.csv'
trends_variant = 'No'

# Import CSVs

In [7]:
# Current Path
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '/financial_data'
print(path)

csvfiles = glob.glob(path + "/*.csv")

# Creating a dataframe for each CSV file
dfs = [pd.read_csv(file) for file in csvfiles]

C:\FYP - Luke Bezzina\Code\preprocessingHistoricalData/financial_data


# Import Google Trends Adjusted dataset

In [8]:
# Current Path
trends_path = ROOT_DIR + '/trends/'
print(trends_path)

# Creating a dataframe for trends
google_df = pd.read_csv(trends_path + trend_file_name)

C:\FYP - Luke Bezzina\Code\preprocessingHistoricalData/trends/


NameError: name 'hbh' is not defined

# New Dataframe

In [ ]:
processed_df_coll = {}

# Pre-Process

In [ ]:
prev_row = None
# Up to 0.2% deviation from close price from day before is considered Neutral movement
neutral_percentage = 0.002

for df in dfs:
    processed_df = pd.DataFrame(columns = ['Date', 'OpenPrice', 'ClosePrice', 'Trend',
                                           'VolumeTrend', 'Volatility', 'PriceDirection'])

    # Creating additional feature columns for each dataframe
    iterables = df.itertuples(index=True, name='Pandas')
    n = df.columns.get_loc('Name') # Name column in df
    df_name = df.iat[1, n] # Name of equity/etf

    print(df_name)

    # Iterating all rows in dataframe
    # Starting from trend_days - to allow for trends going back to trend_days days
    # Ending at end day - 2 to allow for day + 2 classification label
    for i in range(trend_days, len(df) - 2 ):
        open = df.columns.get_loc('Open')
        close = df.columns.get_loc('Close')
        volume = df.columns.get_loc('Volume')
        df_date = df.columns.get_loc('Date')

        # Prices per row
        open_price = df.iat[i-1, open]
        close_price = df.iat[i-1, close]

        # Price Trend Handling
        price_today = df.iat[i + 0, close]
        price_tomorrow = df.iat[i + 1, close]
        price_2days_after = df.iat[i + 2, close]

        close_price_3d = (price_today + price_tomorrow + price_2days_after)/3
        prev_close_price = df.iat[i - 1, close]
        price_change = close_price_3d - prev_close_price

        if price_change > (close_price_3d * neutral_percentage):
            direction = 'Positive'
        elif price_change < 0 and abs(price_change) > (close_price_3d * neutral_percentage):
            direction = 'Negative'
        else:
            direction = 'Neutral'

        # Price Difference Handling
        pds = {}
        initial_close = df.iat[i - (trend_days + 1), close]

        for d in range(1, trend_days + 1):
            key = "d"+ str(d) +"_pd"
            value = df.iat[i - d, close] - initial_close
            pds[key] = value

        # Volume Difference Handling
        vpds = {}
        initial_vol = df.iat[i - (trend_days + 1), volume] + 1

        for vd in range(1, trend_days + 1):
            key = "vd"+ str(vd) +"_pd"
            value = (df.iat[i - vd, volume] / initial_vol)
            vpds[key] = value

        date_ts = df.iat[i,df_date]

        # volatility of prices window
        volatility = np.std(list(pds.values()))

        # finding trends of close and volume in terms of gradient (slope)
        polyfit_x_axis = list(range(0, trend_days))
        test_list = list(pds.values())
        slope, intercept = np.polyfit(polyfit_x_axis, list(pds.values()), 1)
        slopeVol, interceptVol = np.polyfit(polyfit_x_axis, list(vpds.values()), 1)

        new_row = {'Date':date_ts, 'OpenPrice':open_price, 'ClosePrice':close_price,
                   'Trend':slope, 'VolumeTrend':slopeVol, 'Volatility':volatility,
                   'PriceDirection':direction}

        #append row to the dataframe
        processed_df = processed_df.append(new_row, ignore_index=True)

    #pd.concat([processed_df, google_df], axis=1)
    processed_df = pd.merge(processed_df,google_df,left_on='Date',right_on='Date')
    # Re-sorting PriceDirection column
    label = processed_df.pop('PriceDirection')
    processed_df['PriceDirection'] = label

    print(processed_df)
    processed_df_coll[df_name] = processed_df


# Data Export

In [ ]:
for key, df in processed_df_coll.items():
    df.to_csv(ROOT_DIR+'\\LSTM\\'+key+"_"+ str(trend_days) + "dtrend_google_trd" + trends_variant + ".csv", index=False)

print("Export Complete!")
